In [1]:
import pandas as pd

In [2]:
gap_train= pd.read_pickle('./temp_result/train_kaggle_processed')
gap_test= pd.read_pickle('./temp_result/test_kaggle_processed')
gap_valid= pd.read_pickle('./temp_result/valid_kaggle_processed')

In [3]:
def label(A,B):
    if A is True:
        return 0
    if B is True:
        return 1
    return 2

In [43]:
gap_train.tail()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,...,B_pos,pron_pos,A_idx,B_idx,pron_idx,A_vector,B_vector,pron_vector,seq_length,label
1995,test-619,"Fullerton's book, Jane Austen and Crime (2004)...",her,146,Fullerton,0,False,Jane Austen,18,True,...,0.025806,0.174194,"[1, 2]","[7, 8, 9, 10]",[34],"[[0.5251928, -1.0891725, -0.38272008, -0.15374...","[[0.12799127, 0.025941074, -1.1145829, 0.20098...","[[0.047446743, 0.28261346, -0.27585685, -0.506...",38,1
1996,test-353,At a popular bar frequented by fashion magazin...,she,108,Amanda,67,False,Betty,79,True,...,0.093333,0.133333,"[15, 16]","[18, 19]",[26],"[[0.09754669, -0.024278898, -0.15501401, 0.017...","[[0.00817636, 0.6316583, -0.17376839, 0.220072...","[[-0.9807809, 0.6089577, 0.37208742, -0.193520...",36,1
1997,test-1735,"Child's friend, Harriet Winslow Sewall, arrang...",her,87,Harriet Winslow Sewall,16,False,Child,49,True,...,0.092784,0.154639,"[6, 7, 8, 9, 10, 11, 12]",[15],[22],"[[-0.3488823, -0.16492517, 0.014858797, 0.1704...","[[0.047502562, -0.7228153, 0.430767, 0.2394658...","[[-0.7793019, 0.5476636, 0.0666106, -0.1724950...",26,1
1998,test-1856,Lucy struggles with being nice and tries to ge...,her,75,Lucy,0,True,Schroeder,48,False,...,0.102273,0.159091,"[1, 2, 3]","[12, 13, 14, 15, 16]",[22],"[[-0.10890515, -1.071117, -0.1447686, 0.099091...","[[-0.6169741, 0.13752665, 0.14070973, -0.37765...","[[-0.6348102, 0.29169956, -0.04598154, -0.3830...",26,0
1999,test-27,Kathleen first appears when Theresa visits her...,her,43,Kathleen,0,True,Theresa,28,False,...,0.057971,0.086957,"[1, 2, 3]","[7, 8]",[10],"[[-0.17657226, -0.62809116, -0.09252052, 0.296...","[[-0.1495751, -0.03572896, -0.070684716, -0.38...","[[-0.4388417, -0.14650187, 0.09456193, -0.0060...",19,0


In [5]:
import torch
import torch.nn as nn

In [6]:
gap_train["seq_length"] = gap_train.vector.map(lambda x: x.shape[0])
gap_valid["seq_length"] = gap_valid.vector.map(lambda x: x.shape[0])
gap_test["seq_length"] = gap_test.vector.map(lambda x: x.shape[0])

In [7]:
gap_train = gap_train.sort_values(by=['seq_length'], ascending=False)
gap_valid = gap_valid.sort_values(by=['seq_length'], ascending=False)
gap_test = gap_test.sort_values(by=['seq_length'], ascending=False)

In [8]:
gap_train = gap_train.reset_index(drop = True)
gap_valid = gap_valid.reset_index()
gap_test = gap_test.reset_index()

In [9]:
gap_train["label"] = gap_train.apply(lambda x:label(x["A-coref"],x["B-coref"]),axis = 1)
#gap_test["label"] = gap_test.apply(lambda x:label(x["A-coref"],x["B-coref"]),axis = 1)
gap_valid["label"] = gap_valid.apply(lambda x:label(x["A-coref"],x["B-coref"]),axis = 1)

In [10]:
def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        #print ("scores")
        #print (scores.shape)
        #print ("scores")
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

In [11]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
            #print ("inside")
            #print (mask.size())
            #print ("inside")
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k 
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        
        # 2) Apply attention on all the projected vectors in batch. 
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        
        # 3) "Concat" using a view and apply a final linear. 
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

In [12]:
class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

In [13]:
class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

In [14]:
class Encoder(nn.Module):
    "Core encoder is a stack of N layers"
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        #print (x.size())
        #print (mask.size())
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

In [15]:
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

In [16]:
class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [17]:
import copy
c = copy.deepcopy

def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

attn = MultiHeadedAttention(h = 8, d_model = 1024)
ff = PositionwiseFeedForward(d_model = 1024, d_ff = 1024, dropout = 0.5)
enc = Encoder(EncoderLayer(size = 1024, self_attn = c(attn),feed_forward = c(ff), dropout = 0.5), N = 2)

In [18]:
import numpy as np
import math
import torch as F

In [19]:
def get_batch(df,start,batch_size,dim = 1024,test = False):
    num = df.shape[0]
    dim = df.vector.iloc[0].shape[1]
    #print (dim)
    end = min(num,start + batch_size)
    batch_data = df.iloc[start:end,:]
    max_seq_length = df.loc[start,"seq_length"]
    if max_seq_length<32:
        max_seq_length = 32
    padded = np.zeros((end-start,max_seq_length,dim))
    sentence_vector = list(batch_data.vector)
    for i,v in enumerate(sentence_vector):
        padded[i,0:v.shape[0],:] = v 
    if test is False:
        label = batch_data.label
    else:
        label = 0
    A_idx = list(batch_data.A_idx)
    B_idx = list (batch_data.B_idx)
    pron_idx = list(batch_data.pron_idx)
    mask_list  = []
    for i,(a,b,p) in enumerate(zip(A_idx,B_idx,pron_idx)):
        mask = np.zeros((3,max_seq_length))
        mask[0,A_idx[i]] = 1.0
        mask[1,B_idx[i]] = 1.0
        mask[2,pron_idx[i]] = 1.0
        mask_list.append(mask)
    return torch.Tensor(padded),torch.LongTensor(np.array(label)),torch.Tensor(np.array(mask_list))

In [20]:
import torchvision.models as models
class Flatten(torch.nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()
    def forward(self, x):
        N=list(x.size())[0]
        x = x.contiguous()
        x = x.view(N,-1)
        return x

class CNN(nn.Module):
    "Construct a layernorm module (See citation for details)."
    def __init__(self, enc):
        super(CNN, self).__init__()
        self.enc = enc
        self.cnn = nn.Sequential(
                 Flatten(),
                 nn.Linear(3*1024,3)

)
    def forward(self, padded, mask_list):
        mask = (padded.mean(dim = -1) != 0).unsqueeze(-2)
        self.out = enc(padded,mask)
        self.out = self.out.unsqueeze(-3)
        self.out = padded.unsqueeze(-3)
        mask_list = mask_list.unsqueeze(-1)

        self.out = (self.out*mask_list).mean(dim = 2)
        #print (self.out.size())
        self.cnn_out = self.cnn(self.out)
        return self.cnn_out

In [21]:
cnn_model = CNN(enc)
# This was important from their code. 
# Initialize parameters with Glorot / fan_avg.
for p in cnn_model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
cnn_model.cuda()
None

In [22]:
loss_fn = torch.nn.CrossEntropyLoss(reduction='mean')
loss_fn.cuda()
class NoamOpt:
    "Optim wrapper that implements rate."
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0
        
    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self, step = None):
        "Implement `lrate` above"
        if step is None:
            step = self._step
        return self.factor * \
            (self.model_size ** (-0.5) *
            min(step ** (-0.5), step * self.warmup ** (-1.5)))
    def zero_grad(self):
        self.optimizer.zero_grad()
        
def get_std_opt(model):
    return NoamOpt(1024, 2, 4000,
            torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))
optimizer = get_std_opt(cnn_model)

In [23]:
EPOCH = 30
BATCH_SIZE = 8
start = 0
data_num = 2000
for e in range(EPOCH):
    start = 0
    while start <= data_num-1:
        padded, label, mask_list = get_batch(gap_train,start,BATCH_SIZE)
        padded = padded.cuda()
        label = label.cuda()
        mask_list = mask_list.cuda()
        cnn_model.train()
        start = start + BATCH_SIZE
        cnn_out = cnn_model(padded,mask_list)
        cnn_out = cnn_out.view(-1,3)
        #print (label.size())
        cnn_out = torch.nn.Softmax(dim = -1)(cnn_out)
        loss = loss_fn(cnn_out,label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print ("loss at the end of epoch "+str(e))
    print (loss.item())

    

loss at the end of epoch 0
1.092016577720642
loss at the end of epoch 1
1.0461927652359009
loss at the end of epoch 2
0.9984692335128784
loss at the end of epoch 3
0.9573780298233032
loss at the end of epoch 4
0.9219728112220764
loss at the end of epoch 5
0.8905549049377441
loss at the end of epoch 6
0.862732470035553
loss at the end of epoch 7
0.8383117318153381
loss at the end of epoch 8
0.8170197606086731
loss at the end of epoch 9
0.7983737587928772
loss at the end of epoch 10
0.781829833984375
loss at the end of epoch 11
0.7669552564620972
loss at the end of epoch 12
0.7535051107406616
loss at the end of epoch 13
0.7414067387580872
loss at the end of epoch 14
0.730687141418457
loss at the end of epoch 15
0.721389651298523
loss at the end of epoch 16
0.7138261795043945
loss at the end of epoch 17
0.708006739616394
loss at the end of epoch 18
0.703471839427948
loss at the end of epoch 19
0.6999039649963379
loss at the end of epoch 20
0.6970652937889099
loss at the end of epoch 21
0.

In [24]:
def eval_dataset():
    start = 0
    cnn_out_list = []
    while start <= data_num-1:
        padded, label, mask_list = get_batch(gap_test,start,BATCH_SIZE,test = True)
        padded = padded.cuda()
        mask_list = mask_list.cuda()
        cnn_model.eval()
        start = start + BATCH_SIZE
        cnn_out = cnn_model(padded,mask_list)
        cnn_out = cnn_out.view(-1,3)
        cnn_out = torch.nn.Softmax(dim = 1)(cnn_out)
        cnn_out_list.append(cnn_out.cpu().data.numpy())
    return np.concatenate(cnn_out_list,axis = 0)

In [50]:
cnn_out_list = eval_dataset()

In [52]:
cnn_out_list

array([[3.4601706e-01, 6.3741124e-01, 1.6571701e-02],
       [4.4428149e-01, 5.5370706e-01, 2.0114044e-03],
       [4.9062228e-01, 4.8169231e-01, 2.7685393e-02],
       ...,
       [6.4839027e-03, 9.9351609e-01, 2.0165412e-09],
       [7.3773915e-01, 2.6226091e-01, 7.4873725e-11],
       [2.3239160e-01, 7.6760650e-01, 1.9149309e-06]], dtype=float32)

In [44]:
cnn_out_list = pd.DataFrame(np.array(cnn_out_list),index = gap_test["index"])

In [46]:
cnn_out_list = cnn_out_list.sort_index()
cnn_out_list.head()

,0,1,2
index,,,
0,0.487733,0.512266,8.340291e-07
1,0.885138,0.114859,3.337658e-06
2,0.112824,0.886892,2.839654e-04
3,0.185579,0.814405,1.509802e-05
4,0.121560,0.878009,4.306988e-04


In [48]:
sub_df = pd.read_csv("./test_and_submit/sample_submission_stage_1.csv")
sub_df.loc[:, ['A','B','NEITHER']] = cnn_out_list.values

sub_df.head()

,ID,A,B,NEITHER
0,development-1,0.487733,0.512266,8.340291e-07
1,development-2,0.885138,0.114859,3.337658e-06
2,development-3,0.112824,0.886892,2.839654e-04
3,development-4,0.185579,0.814405,1.509802e-05
4,development-5,0.121560,0.878009,4.306988e-04


In [49]:
import datetime
sub_df.to_csv("./test_and_submit/submission+model+transformer@"+str(datetime.datetime.now())+".csv", index=False)

In [ ]:
import datetime
sub_df.to_csv("./test_and_submit/submission+model+mlp@"+str(datetime.datetime.now())+".csv", index=False)